# آليات الانتباه والنماذج المحولة

أحد العيوب الرئيسية للشبكات العودية هو أن جميع الكلمات في تسلسل معين لها نفس التأثير على النتيجة. يؤدي هذا إلى أداء غير مثالي مع نماذج LSTM القياسية للترميز وفك الترميز في مهام تحويل التسلسل إلى تسلسل، مثل التعرف على الكيانات المسماة والترجمة الآلية. في الواقع، غالبًا ما يكون لبعض الكلمات في تسلسل الإدخال تأثير أكبر على المخرجات المتسلسلة مقارنة بغيرها.

لنأخذ نموذج تحويل التسلسل إلى تسلسل، مثل الترجمة الآلية. يتم تنفيذه باستخدام شبكتين عوديتين، حيث تقوم شبكة واحدة (**المشفّر**) بضغط تسلسل الإدخال إلى حالة مخفية، وتقوم الأخرى، **المفكّك**، بفك هذه الحالة المخفية إلى النتيجة المترجمة. المشكلة في هذا النهج هي أن الحالة النهائية للشبكة ستواجه صعوبة في تذكر بداية الجملة، مما يؤدي إلى ضعف جودة النموذج في الجمل الطويلة.

**آليات الانتباه** توفر وسيلة لوزن التأثير السياقي لكل متجه إدخال على كل توقع إخراج للشبكة العودية. يتم تنفيذ ذلك من خلال إنشاء اختصارات بين الحالات الوسيطة لشبكة الإدخال العودية وشبكة الإخراج العودية. بهذه الطريقة، عند توليد رمز الإخراج $y_t$، سنأخذ في الاعتبار جميع الحالات المخفية للإدخال $h_i$، مع معاملات وزن مختلفة $\alpha_{t,i}$.

![صورة توضح نموذج الترميز/فك الترميز مع طبقة انتباه إضافية](../../../../../translated_images/ar/encoder-decoder-attention.7a726296894fb567.webp)
*نموذج الترميز-فك الترميز مع آلية الانتباه الإضافية في [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)، مقتبس من [هذا المقال](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

مصفوفة الانتباه $\{\alpha_{i,j}\}$ تمثل الدرجة التي تلعب بها كلمات الإدخال دورًا في توليد كلمة معينة في تسلسل الإخراج. أدناه مثال على مثل هذه المصفوفة:

![صورة توضح محاذاة نموذج RNNsearch-50، مأخوذة من Bahdanau - arviz.org](../../../../../translated_images/ar/bahdanau-fig3.09ba2d37f202a6af.webp)

*الصورة مأخوذة من [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (الشكل 3)*

آليات الانتباه مسؤولة عن الكثير من التقدم الحالي أو القريب من الحالي في معالجة اللغة الطبيعية. ومع ذلك، فإن إضافة الانتباه تزيد بشكل كبير من عدد معلمات النموذج، مما أدى إلى مشكلات في التوسع مع الشبكات العودية. أحد القيود الرئيسية لتوسيع الشبكات العودية هو أن الطبيعة العودية للنماذج تجعل من الصعب تجميع وتوازي التدريب. في الشبكات العودية، يجب معالجة كل عنصر من عناصر التسلسل بترتيب متسلسل، مما يعني أنه لا يمكن توازيته بسهولة.

اعتماد آليات الانتباه مع هذا القيد أدى إلى إنشاء نماذج المحولات التي تمثل الآن أحدث ما توصلت إليه التكنولوجيا، والتي نعرفها ونستخدمها اليوم مثل BERT وOpenGPT3.

## نماذج المحولات

بدلاً من تمرير سياق كل توقع سابق إلى خطوة التقييم التالية، تستخدم **نماذج المحولات** **الترميزات الموضعية** والانتباه لالتقاط سياق الإدخال ضمن نافذة نصية محددة. الصورة أدناه توضح كيف يمكن للترميزات الموضعية مع الانتباه التقاط السياق ضمن نافذة معينة.

![صورة متحركة توضح كيفية إجراء التقييمات في نماذج المحولات.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

نظرًا لأن كل موضع إدخال يتم تعيينه بشكل مستقل إلى كل موضع إخراج، يمكن للمحولات التوازي بشكل أفضل من الشبكات العودية، مما يتيح نماذج لغوية أكبر وأكثر تعبيرًا. يمكن لكل رأس انتباه أن يُستخدم لتعلم علاقات مختلفة بين الكلمات، مما يحسن مهام معالجة اللغة الطبيعية.

**BERT** (تمثيلات الترميز ثنائية الاتجاه من المحولات) هو شبكة محولات متعددة الطبقات كبيرة جدًا تحتوي على 12 طبقة في *BERT-base*، و24 طبقة في *BERT-large*. يتم تدريب النموذج أولاً على مجموعة نصوص كبيرة (ويكيبيديا + كتب) باستخدام تدريب غير خاضع للإشراف (توقع الكلمات المحجوبة في الجملة). أثناء التدريب الأولي، يكتسب النموذج مستوى كبيرًا من فهم اللغة يمكن الاستفادة منه لاحقًا مع مجموعات بيانات أخرى باستخدام التخصيص الدقيق. تُعرف هذه العملية باسم **التعلم بالنقل**.

![صورة من http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ar/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

هناك العديد من التعديلات على بنية المحولات، بما في ذلك BERT، وDistilBERT، وBigBird، وOpenGPT3، والمزيد، التي يمكن تخصيصها. حزمة [HuggingFace](https://github.com/huggingface/) توفر مستودعًا لتدريب العديد من هذه البنى باستخدام PyTorch.

## استخدام BERT لتصنيف النصوص

دعونا نرى كيف يمكننا استخدام نموذج BERT المدرب مسبقًا لحل مهمتنا التقليدية: تصنيف التسلسل. سنقوم بتصنيف مجموعة بيانات AG News الأصلية.

أولاً، دعونا نحمل مكتبة HuggingFace ومجموعة البيانات الخاصة بنا:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


نظرًا لأننا سنستخدم نموذج BERT المدرب مسبقًا، سنحتاج إلى استخدام مُجزئ محدد. أولاً، سنقوم بتحميل مُجزئ مرتبط بنموذج BERT المدرب مسبقًا.

تحتوي مكتبة HuggingFace على مستودع للنماذج المدربة مسبقًا، والتي يمكنك استخدامها فقط عن طريق تحديد أسمائها كوسائط في دوال `from_pretrained`. سيتم تنزيل جميع الملفات الثنائية المطلوبة للنموذج تلقائيًا.

ومع ذلك، في بعض الأحيان قد تحتاج إلى تحميل نماذجك الخاصة، وفي هذه الحالة يمكنك تحديد الدليل الذي يحتوي على جميع الملفات ذات الصلة، بما في ذلك معلمات المُجزئ، ملف `config.json` الذي يحتوي على معلمات النموذج، الأوزان الثنائية، وما إلى ذلك.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

يحتوي كائن `tokenizer` على وظيفة `encode` التي يمكن استخدامها مباشرةً لترميز النص:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

ثم، دعونا ننشئ المكررات التي سنستخدمها أثناء التدريب للوصول إلى البيانات. نظرًا لأن BERT يستخدم وظيفة الترميز الخاصة به، سنحتاج إلى تعريف وظيفة تعبئة مشابهة لـ `padify` التي قمنا بتعريفها سابقًا:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

في حالتنا، سنستخدم نموذج BERT المدرب مسبقًا يسمى `bert-base-uncased`. دعنا نقوم بتحميل النموذج باستخدام حزمة `BertForSequenceClassfication`. هذا يضمن أن النموذج لدينا يحتوي بالفعل على البنية المطلوبة للتصنيف، بما في ذلك المصنف النهائي. سترى رسالة تحذير تشير إلى أن أوزان المصنف النهائي غير مهيأة، وأن النموذج سيحتاج إلى تدريب مسبق - وهذا أمر طبيعي تمامًا، لأنه بالضبط ما نحن على وشك القيام به!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

الآن نحن مستعدون لبدء التدريب! نظرًا لأن BERT مُدرّب مسبقًا، نرغب في البدء بمعدل تعلم صغير جدًا حتى لا نتسبب في تدمير الأوزان الأولية.

كل العمل الشاق يتم بواسطة نموذج `BertForSequenceClassification`. عندما نستدعي النموذج على بيانات التدريب، فإنه يعيد كلًا من الخسارة ومخرجات الشبكة للدفعة المصغرة المدخلة. نستخدم الخسارة لتحسين المعاملات (`loss.backward()` يقوم بالتمرير العكسي)، و`out` لحساب دقة التدريب من خلال مقارنة التصنيفات المحصلة `labs` (المحسوبة باستخدام `argmax`) مع التصنيفات المتوقعة `labels`.

للتحكم في العملية، نقوم بتجميع الخسارة والدقة على مدى عدة تكرارات، ونعرضها كل `report_freq` دورة تدريبية.

من المحتمل أن يستغرق هذا التدريب وقتًا طويلًا، لذا نقوم بتحديد عدد التكرارات.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


يمكنك أن تلاحظ (خصوصًا إذا زدت عدد التكرارات وانتظرت لفترة كافية) أن تصنيف BERT يمنحنا دقة جيدة جدًا! يعود ذلك إلى أن BERT يفهم بالفعل بنية اللغة بشكل جيد، وكل ما نحتاجه هو ضبط المصنف النهائي. ومع ذلك، نظرًا لأن BERT نموذج كبير، فإن عملية التدريب بأكملها تستغرق وقتًا طويلًا وتتطلب قوة حوسبة كبيرة! (وحدة معالجة رسومات GPU، ويفضل أن تكون أكثر من واحدة).

> **ملاحظة:** في مثالنا، كنا نستخدم أحد أصغر نماذج BERT المدربة مسبقًا. هناك نماذج أكبر من المحتمل أن تحقق نتائج أفضل.


## تقييم أداء النموذج

الآن يمكننا تقييم أداء النموذج على مجموعة البيانات الخاصة بالاختبار. حلقة التقييم تشبه إلى حد كبير حلقة التدريب، ولكن يجب ألا ننسى تبديل النموذج إلى وضع التقييم عن طريق استدعاء `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## النقاط الرئيسية

في هذه الوحدة، رأينا مدى سهولة استخدام نموذج لغة مدرب مسبقًا من مكتبة **transformers** وتكييفه مع مهمة تصنيف النصوص الخاصة بنا. وبالمثل، يمكن استخدام نماذج BERT لاستخراج الكيانات، والإجابة على الأسئلة، والمهام الأخرى في معالجة اللغة الطبيعية.

تمثل نماذج المحولات (Transformers) أحدث ما توصلت إليه التكنولوجيا في مجال معالجة اللغة الطبيعية، وفي معظم الحالات، يجب أن تكون الحل الأول الذي تبدأ بتجربته عند تنفيذ حلول مخصصة لمعالجة اللغة الطبيعية. ومع ذلك، فإن فهم المبادئ الأساسية للشبكات العصبية المتكررة التي تمت مناقشتها في هذه الوحدة أمر بالغ الأهمية إذا كنت ترغب في بناء نماذج عصبية متقدمة.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
